In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 10

import warnings
warnings.filterwarnings("ignore")

import os, sys
import pandas as pd
import pickle
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

Autosaving every 10 seconds


In [2]:
gpu_id='gpu3'
import os
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"

from utils.w2v_initialization import *
from utils.models import *
from utils.extentions import *

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Activation, Dense, Dropout, LSTM, Merge
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using Theano backend.
Using gpu device 3: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


In [4]:
LEMMA_IND = 7
POS_IND = 9

SELECTED_FEATURES = "/home/sag/syntactic/src_putin/grant/data/processed/selected_features/"
word_to_ind_mapping = pickle.load(open(SELECTED_FEATURES+"word_to_ind_mapping.pkl", 'rb'))
model_with_w2v = load_keras_model(SELECTED_FEATURES+"model_with_w2v_init.pkl")

In [5]:
matrix_w2v = np.load(SELECTED_FEATURES+"matrix_updated.npy")
matrix = np.delete(np.load(SELECTED_FEATURES+"matrix.npy"), (71079), axis=0)

target = OneHotEncoder(sparse=False).fit_transform(np.load(SELECTED_FEATURES+"y.npy").reshape(-1, 1) - 1)
target = np.delete(target, (71079), axis=0)
print(matrix.shape, target.shape)

#X_train, X_val, X_test, y_train, y_val, y_test = matrix_train_val_test_split(matrix, target, number_of_samples_to_use=10000)
X_train, X_val, X_test, y_train, y_val, y_test = matrix_train_val_test_split(matrix, target)
X_train_w2v, X_val_w2v, X_test_w2v, y_train_w2v, y_val_w2v, y_test_w2v = matrix_train_val_test_split(matrix_w2v, target)

((1423937, 13, 12), (1423937, 125))
((1153388, 13, 12), (128155, 13, 12), (142394, 13, 12), (1153388, 125), (128155, 125), (142394, 125))
((1153388, 13, 13), (128155, 13, 13), (142394, 13, 13), (1153388, 125), (128155, 125), (142394, 125))


In [6]:
string = "4.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t2.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t1.0\t2.0\t2.0\t2.0\t2.0\t0.0\t0.0\t0.0\t1.0\t2.0\t2.0\t7.0\t7.0\t2.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t2.0\t2.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t2.0\t2.0\t2.0\t1.0\t1.0\t2.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t195.0\t1773.0\t0.0\t2253.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t195.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t2.0\t4.0\t2.0\t0.0\t0.0\t0.0\t4.0\t2.0\t2.0\t3.0\t3.0\t2.0\t0.0\t0.0\t4.0\t0.0\t0.0\t0.0\t5.0\t4.0\t3.0\t2.0\t1.0\t0.0\t0.0\t0.0\t15.0\t16.0\t17.0\t18.0\t19.0\t4.0\t0.0\t0.0\t13.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t2.0\t2.0\t1.0\t2.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t2.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t3.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t3.0\t1.0\t1.0\t1.0\t0.0\t0.0\t2.0\t0.0\t0.0\t0.0\t1.0\t1.0\t2.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t3.0\t3.0\t1.0\t1.0\t1.0\t0.0\t0.0\t2.0\t0.0\t0.0\t0.0\t0.0\t163.0\t454.0\t2505.0\t440.0\t0.0\t0.0\t0.0\t0.0\t1868.0\t0.0\t188.0\t0.0\t163.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t2.0\t1.0\t2.0\t0.0\t0.0\t0.0\t1.0\t3.0\t2.0\t1.0\t1.0\t2.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t1.0\t9.0\t3.0\t5.0\t2.0\t0.0\t0.0\t0.0\t1.0\t3.0\t3.0\t5.0\t1.0\t9.0\t0.0\t0.0\t3.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t2.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t2.0\t0.0159\t0.0729\t0.1121\t-0.0178\t0.0184\t-0.0069\t-0.0325\t0.076\t0.05\t-0.0626\t0.0326\t-0.0448\t-0.0035\t0.0508\t0.0187\t0.0911\t0.02\t0.047\t-0.0846\t0.0652\t-0.0175\t-0.0284\t0.0283\t0.055\t0.0736\t0.1012\t0.0628\t0.0886\t0.0222\t0.1382\t0.0322\t0.0204\t0.0464\t0.022\t0.0399\t-0.0105\t0.0348\t-0.0377\t0.0301\t0.0189\t-0.0199\t-0.1969\t0.0305\t0.0702\t-0.0263\t0.0065\t0.1031\t-0.0307\t-0.0418\t0.0271\t-0.0345\t0.0585\t-0.1299\t0.0476\t-0.0833\t0.11\t-0.0444\t0.0181\t0.0749\t0.0138\t-0.0127\t-0.0064\t-0.0425\t-0.0216\t0.0135\t-0.0125\t0.053\t0.063\t-0.0576\t-0.0376\t0.0607\t-0.0298\t0.058\t0.0846\t0.0488\t0.0455\t-0.0894\t-0.1224\t-0.0286\t0.0762\t0.0204\t0.1149\t1.0E-4\t-0.0441\t0.0937\t0.0117\t-0.1359\t0.0506\t0.0075\t0.0274\t0.1041\t-0.0548\t0.0218\t-0.0472\t-0.0343\t-0.031\t0.0171\t-0.1141\t-0.0693\t-0.0223\t-0.04\t-0.0568\t-0.0313\t0.0176\t-0.0053\t-0.1095\t-0.0296\t0.0176\t0.0367\t-0.055\t0.008\t-0.0943\t0.0054\t-0.0521\t-0.0849\t0.0444\t-0.0639\t0.0386\t0.1105\t-0.0272\t0.0265\t0.0268\t-0.0329\t0.0205\t0.0368\t0.0199\t-0.0102\t0.1121\t0.0518\t-0.0232\t-0.0459\t-0.0501\t-0.013\t-0.0628\t0.0638\t0.0917\t-0.0467\t0.0147\t-0.0746\t-0.0124\t-0.2238\t0.0341\t-0.0444\t0.0381\t0.0251\t-0.0175\t-0.0039\t-0.043\t-0.0703\t-0.0614\t-0.0699\t0.0018\t-0.077\t0.0342\t0.0237\t0.0187\t-0.0231\t-0.1294\t-0.0397\t-0.023\t-0.0316\t0.0637\t0.0607\t0.0181\t0.0862\t0.067\t-0.0356\t-0.0213\t0.0538\t0.0112\t0.0037\t0.0269\t0.0121\t-0.0227\t-0.0741\t-0.0746\t0.0176\t-0.0249\t-0.0091\t0.0533\t-0.0783\t0.0293\t-0.0047\t0.0518\t-0.0763\t-0.0216\t-0.0792\t-0.0039\t-0.0236\t0.0314\t-0.051\t0.1685\t-0.0099\t-0.0207\t-0.0111\t0.0233\t0.1063\t0.0641\t0.0755\t-0.0067\t0.0295\t-0.0373\t0.0209\t0.0581\t-0.0563\t0.0048\t-0.0153\t-0.0698\t0.0433\t-0.1095\t-0.0202\t-6.0E-4\t0.0064\t-0.0907\t-0.04\t-0.0181\t0.032\t0.0019\t-0.0285\t0.0339\t-0.0638\t-0.0557\t-0.099\t0.0096\t0.0216\t-0.061\t-0.0211\t-0.0805\t-0.0148\t-0.0264\t-0.0379\t-0.0313\t0.0327\t0.0254\t0.0502\t-0.0662\t-0.0486\t-0.0852\t0.0475\t-0.0216\t0.0084\t0.0494\t-0.0277\t-0.0253\t0.0234\t0.0044\t-0.0011\t-0.0335\t0.0746\t0.0038\t-0.0431\t-0.0049\t-0.0632\t0.0241\t0.0277\t-0.0292\t0.0457\t0.0019\t0.0966\t-0.0464\t0.0\t0.0133\t-0.0514\t0.0662\t0.1149\t0.0416\t0.0107\t0.0384\t-0.0122\t0.0653\t-0.0147\t-0.0672\t0.0862\t0.0357\t-0.0184\t0.0408\t-0.0214\t0.0472\t-0.0026\t-0.0081\t-0.0237\t-0.0499\t0.0438\t0.0189\t0.0212\t0.0647\t-0.0323\t-0.0855\t-0.0714\t0.0201\t0.0307\t-0.0937\t0.1298\t-0.0132\t-0.0228\t-0.0383\t0.0162\t0.0427\t-0.0538\t0.0553\t0.0309\t-0.0273\t0.011\t0.0537\t0.01\t0.0552\t0.0152\t0.0544\t-0.0893\t-0.0589\t0.0355\t-0.0382\t0.1036\t0.0017\t0.0667\t0.0607\t-0.0613\t-0.0475\t0.0733\t0.0689\t0.1017\t-0.0128\t-0.0218\t-0.0147\t-0.0652\t-0.0826\t0.1008\t-0.0396\t-0.0404\t-0.0016\t-0.1275\t-0.0769\t-0.0969\t0.0664\t0.0414\t0.0153\t-0.0205\t-0.007\t0.0071\t0.0557\t0.0461\t0.0435\t-0.0049\t-0.0187\t-0.0115\t-0.0301\t-0.0174\t0.0494\t-0.0279\t0.085\t0.0976\t0.0736\t0.0315\t-0.0477\t-0.0277\t0.1026\t0.0101\t0.0075\t-0.0323\t-0.0261\t0.0803\t-3.0E-4\t-0.0511\t-0.0807\t0.0413\t0.0225\t-0.0251\t0.0275\t-0.0011\t0.0735\t-0.0506\t-0.0463\t0.0251\t0.0472\t-0.0423\t-0.0133\t-0.0414\t0.1984\t0.0102\t-0.1003\t-0.0583\t-0.0254\t-0.1\t-0.0251\t-0.0147\t0.0869\t0.0282\t-0.0149\t-0.1257\t-0.0202\t-0.0319\t-0.0046\t0.1024\t-0.0503\t0.0248\t-0.0392\t-0.0769\t0.0394\t-0.0032\t-0.0887\t-0.034\t-0.0555\t-0.004\t6.0E-4\t0.0332\t0.0381\t0.0528\t0.0215\t-0.0927\t0.0229\t-0.0433\t0.0201\t0.0109\t0.0361\t0.1145\t-0.0018\t-0.1095\t-0.0369\t0.0458\t0.0174\t0.0448\t0.0255\t0.0205\t-0.0704\t-0.0089\t-0.0588\t-0.0579\t0.0816\t0.0203\t-0.0531\t0.0024\t0.0054\t-3.0E-4\t0.0057\t0.0555\t-0.0381\t0.0069\t0.1193\t0.0162\t-0.1147\t0.0436\t0.0274\t-0.0479\t0.0737\t-0.1199\t0.0397\t-0.0063\t6.0E-4\t-0.0032\t-0.0762\t0.0348\t-0.0755\t0.0073\t0.0741\t0.0752\t-0.0061\t-0.0014\t-0.063\t0.0662\t-0.1048\t0.0014\t0.0017\t-0.099\t-0.0049\t-0.0155\t0.0964\t0.0209\t0.0587\t-0.0835\t0.0116\t-0.0044\t-0.0252\t-0.0192\t-0.0969\t0.0446\t-0.0395\t0.0065\t-0.0073\t-0.1333\t0.0594\t-0.0466\t-0.0034\t0.0411\t-0.0745\t0.036\t-0.0537\t0.0966\t0.0981\t-0.019\t0.0105\t0.1192\t-0.0835\t0.0023\t0.0351\t-0.0423\t0.009\t0.0478\t0.0101\t0.0056\t0.0354\t0.0817\t-0.0141\t0.173\t-0.1419\t0.0729\t-0.1381\t0.0332\t0.0604\t-0.0053\t-0.0202\t0.0388\t0.0438\t-0.0324\t0.0057\t-0.0054\t-0.0216\t-0.0237\t0.0355\t-0.0442\t-0.0089\t0.1449\t0.082\t0.0574\t-0.0062\t0.0234\t0.0139\t-0.0469\t-0.0031\t-0.0311\t-0.0229\t0.007\t0.0492\t0.1116\t0.0187\t-0.0426\t0.0313\t0.0204\t0.0225\t0.0226\t0.0286\t-0.051\t0.0073\t0.1328\t-0.1119\t0.0036\t-0.0303\t0.0322\t-0.0186\t0.0312\t0.0953\t-0.0072\t-0.0108\t-0.0586\t0.0013\t0.0225\t-0.0211\t0.0727\t0.0527\t0.1271\t-0.0818\t0.001\t-0.0357\t0.0906\t-0.0229\t-0.0399\t0.0671\t0.0432\t-0.0121\t-0.025\t0.0317\t0.0028\t-0.0524\t-0.0231\t-0.0059\t0.1235\t0.0124\t0.0206\t-0.054\t-0.0794\t0.0025\t-0.0545\t0.0206\t0.0111\t0.0106\t0.0937\t0.1508\t-0.1091\t-0.044\t0.0218\t0.0917\t-0.0297\t0.0943\t0.1153\t-0.0531\t-0.0605\t0.0039\t-0.0217\t-0.1486\t0.0182\t-0.1357\t0.0\t0.0\t0.0\t2.0\t2.0\t2.0\t2.0\t2.0\t0.0\t0.0\t0.0\t1.0\t2.0\t2.0\t2.0\t1.0\t2.0\t0.0\t0.0\t3.0\t9.0\t156.0\t1.0\t1.0\t26.0\t30.0\t25.0\t16.0\t74.0\t37.0\t77.0\t2.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t26.0\t5.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t9.0\t26.0\t0.0\t0.0"
string2 = "0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t2.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t2.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t4.0\t5.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t4.0\t5.0\t6.0\t7.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t1.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t1.0\t3.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t4.0\t5.0\t6.0\t7.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t2.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t2.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t2.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t2.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t2.0\t1.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t2.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t4.0\t5.0\t6.0\t7.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t1.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t2.0\t1.0\t3.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t3.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t4.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t0.0\t0.0\t0.0\t2.0\t1.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t1.0\t0.0\t-0.0237\t0.0445\t0.0294\t-0.0114\t0.0382\t0.0245\t0.0366\t0.0002\t0.0356\t0.0756\t-0.0402\t-0.002\t0.0117\t-0.0321\t-0.032\t0.0273\t0.062\t-0.0398\t-0.0652\t0.016\t-0.0347\t0.1358\t0.056\t-0.0608\t0.0291\t-0.0455\t-0.0546\t0.0364\t-0.042\t0.0507\t0.1149\t-0.0956\t-0.0132\t-0.003\t-0.0224\t-0.0603\t-0.0348\t0.0132\t-0.0104\t0.021\t-0.1075\t0.0891\t-0.0435\t0.0594\t0.0134\t0.0182\t0.0068\t-0.0092\t0.0185\t0.0568\t0.1128\t0.0044\t0.0716\t0.0232\t-0.036\t-0.0577\t0.0732\t0.0312\t0.0164\t-0.0643\t0.0564\t-0.0202\t0.0906\t-0.0462\t0.01\t-0.0703\t0.0415\t0.0093\t-0.0862\t0.0537\t-0.0571\t0.0407\t0.0673\t0.0672\t0.0267\t0.0305\t-0.0796\t-0.0617\t-0.0355\t0.0413\t-0.0221\t-0.0467\t-0.0257\t0.0671\t-0.0938\t0.0291\t-0.0155\t0.0453\t-0.0567\t-0.0901\t0.0671\t0.0062\t-0.0554\t-0.0215\t0.0725\t0.0285\t0.0994\t0.1102\t-0.0267\t-0.022\t0.0067\t-0.0369\t0.0715\t-0.0313\t-0.0187\t0.0273\t-0.1014\t0.0508\t0.0406\t-0.0623\t-0.1217\t-0.1316\t-0.0437\t0.0463\t0.0105\t-0.01\t0.0042\t0.0154\t-0.0255\t0.0398\t-0.0593\t-0.0301\t-0.0107\t0.0003\t-0.0334\t-0.0522\t-0.0202\t-0.0538\t-0.0473\t-0.0809\t-0.0679\t0.1205\t0.0551\t-0.0136\t0.076\t-0.0766\t-0.0477\t0.0927\t-0.0549\t-0.0079\t-0.0138\t0.0347\t0.063\t0.0074\t-0.0349\t0.0423\t0.0306\t0.0568\t-0.1582\t0.1002\t-0.0331\t0.0139\t-0.0289\t-0.0381\t0.0781\t0.0479\t0.1732\t-0.0753\t0.0107\t0.0375\t0.0404\t0.0634\t0.0383\t0.0583\t-0.0564\t0.0789\t-0.0194\t-0.0384\t-0.133\t-0.0451\t-0.0529\t0.0423\t0.041\t0.0564\t0.1056\t-0.1219\t0.0525\t0.0893\t-0.0016\t0.0102\t-0.0568\t0.0124\t-0.045\t0.0132\t0.0396\t-0.0282\t0.0402\t-0.029\t0.0306\t0.0171\t0.0109\t-0.1024\t0.0367\t-0.0087\t0.0018\t0.0471\t0.0617\t0.0219\t-0.0366\t0.0308\t0.0314\t-0.0474\t-0.1647\t-0.0141\t0.004\t-0.0899\t0.0338\t-0.0383\t0.0529\t0.0766\t0.0751\t-0.0482\t-0.0003\t0.0561\t0.0915\t0.0533\t-0.0793\t-0.0831\t-0.0971\t-0.0469\t0.054\t-0.0647\t0.0074\t0.0329\t-0.0242\t-0.0171\t0.0076\t0.0387\t-0.0209\t0.0049\t-0.1325\t-0.0802\t-0.023\t0.0123\t-0.0115\t-0.0654\t-0.0843\t-0.0012\t-0.0132\t0.0453\t0.0529\t-0.0435\t0.0472\t0.0384\t0.0737\t-0.054\t0.0264\t0.0245\t-0.0562\t-0.0413\t0.0188\t0.0\t-0.0707\t-0.0657\t-0.0662\t0.0067\t0.0658\t0.0594\t0.0018\t0.0203\t0.0242\t0.0947\t0.0936\t-0.0249\t0.0026\t0.0499\t-0.1133\t0.0332\t-0.0751\t0.0547\t0.106\t0.0329\t0.0351\t-0.001\t-0.0087\t-0.0845\t0.0407\t0.1185\t-0.0248\t0.046\t0.0279\t-0.0259\t0.0382\t-0.015\t-0.0051\t0.0694\t0.0295\t0.0106\t-0.0583\t-0.0328\t-0.0892\t-0.0189\t0.0488\t-0.0593\t0.1667\t-0.0345\t-0.0394\t-0.0644\t-0.0231\t-0.0687\t0.0756\t-0.0895\t0.0308\t0.0529\t-0.002\t0.0741\t-0.0765\t0.1104\t0.0127\t-0.0873\t-0.0194\t-0.0153\t-0.0024\t-0.015\t-0.0394\t-0.0441\t0.0205\t0.0298\t-0.0427\t0.0195\t-0.0234\t-0.067\t-0.0932\t-0.0416\t-0.0204\t0.0294\t0.0254\t-0.0113\t-0.0281\t-0.0569\t-0.0225\t0.1175\t0.0615\t0.0719\t-0.0858\t-0.0281\t-0.0929\t-0.0769\t0.0493\t0.0863\t0.0065\t0.0292\t-0.0115\t0.1121\t-0.0021\t0.0086\t-0.0196\t-0.0547\t0.0025\t0.0589\t0.0716\t-0.0745\t-0.0718\t-0.0147\t-0.0441\t-0.0604\t-0.0098\t-0.0094\t-0.0299\t-0.0879\t-0.1166\t0.0143\t-0.0446\t-0.0432\t0.0648\t-0.0154\t0.0098\t-0.0197\t0.0121\t-0.0053\t-0.0065\t-0.0327\t-0.0476\t0.0457\t-0.1007\t-0.0925\t-0.0157\t0.0073\t0.1385\t0.0977\t-0.001\t0.0747\t-0.0871\t-0.0914\t0.0618\t-0.0296\t-0.0741\t-0.0521\t0.0282\t-0.0485\t0.0537\t-0.051\t0.0673\t0.0121\t0.015\t0.154\t-0.0591\t0.028\t-0.0268\t0.0954\t-0.0398\t0.0379\t0.0492\t-0.0839\t0.06\t0.0471\t0.0401\t-0.023\t-0.0003\t-0.0701\t-0.0022\t0.0289\t0.0796\t-0.0601\t-0.0804\t0.0738\t-0.0237\t0.1201\t-0.0136\t-0.1338\t-0.053\t0.0334\t-0.0539\t0.0162\t0.0458\t0.0649\t0.1209\t-0.0623\t-0.0947\t-0.1523\t-0.0917\t-0.0106\t-0.0193\t0.0489\t0.0363\t0.017\t-0.0585\t-0.0772\t-0.0737\t-0.0145\t-0.0553\t0.0656\t0.0545\t0.0044\t0.0674\t0.0379\t-0.0489\t-0.0294\t0.0334\t0.021\t-0.0541\t-0.0369\t0.0105\t0.0607\t-0.0045\t0.068\t-0.0867\t-0.0423\t-0.0399\t-0.0635\t0.0231\t-0.195\t0.0544\t-0.0132\t0.0456\t-0.1608\t0.0472\t-0.0161\t-0.0015\t-0.0188\t0.0521\t0.0406\t0.0427\t-0.0554\t0.1189\t-0.0018\t-0.0028\t0.0099\t-0.0409\t0.0261\t0.0032\t0.0096\t0.071\t-0.0058\t-0.0058\t0.0625\t0.0267\t0.0574\t0.0284\t-0.0541\t0.1102\t0.0481\t0.065\t0.0253\t-0.0559\t-0.0264\t-0.0061\t-0.0037\t0.0101\t0.0112\t0.0648\t-0.0481\t-0.031\t0.0511\t-0.0417\t0.0849\t-0.1207\t0.1223\t0.0147\t-0.0384\t-0.0654\t-0.0324\t0.0446\t0.0208\t0.0013\t0.0252\t-0.0807\t-0.0215\t0.0196\t-0.0358\t0.0665\t0.0304\t0.0631\t0.0791\t0.0462\t0.0136\t-0.0871\t-0.0194\t-0.0478\t0.0698\t-0.0039\t0.1004\t0.0195\t0.0792\t0.0439\t-0.0376\t0.0455\t0.0614\t0.057\t0.0352\t-0.0463\t-0.0447\t-0.014\t-0.0362\t-0.0283\t-0.0029\t-0.0765\t-0.0252\t0.1406\t0.0134\t-0.0155\t0.0908\t-0.0799\t0.0062\t0.0427\t-0.0329\t-0.0074\t-0.0925\t-0.038\t0.114\t-0.0915\t0.0687\t0.0131\t-0.0349\t0.0171\t0.0359\t0.0472\t-0.0102\t-0.0324\t0.0504\t-0.0515\t-0.106\t0.0388\t0.0278\t-0.033\t0.0692\t-0.0478\t-0.0159\t0.0608\t0.043\t0.0876\t0.01\t0.0006\t-0.0281\t0.0249\t0.0664\t0.0352\t-0.0713\t0.0045\t0.0156\t-0.0715\t-0.0389\t-0.031\t0.0708\t0.0525\t0.0888\t-0.0612\t0.0764\t-0.0345\t0.0429\t0.0\t0.0\t0.0\t2.0\t2.0\t1.0\t0.0\t0.0\t0.0\t0.0\t0.0\t1.0\t2.0\t1.0\t2.0\t0.0\t0.0\t0.0\t1.0\t0.0\t6.0\t6.0\t1.0\t1.0\t3.0\t4.0\t5.0\t3.0\t4.0\t6.0\t11.0\t2.0\t2.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t1.0\t0.0\t0.0\t0.0\t4.0\t3.0\t0.0\t0.0\t2.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0\t0.0"
print(len(string.split('\t')))
print(len(string2.split('\t')))

1292
1293


In [7]:
def get_matrix_features_by_str_without_w2v(string, matrix_index_path, mapping_input_str_path):
    mapping_input_str = open(mapping_input_str_path).readlines()
    mapping_input_str = [x[:-2] for x in mapping_input_str] # delete \r \n symbols
    
    features_to_values = {k: v for k, v in zip(mapping_input_str, string.split('\t'))}
    
    sliced_matrix_index = pd.read_csv(matrix_index_path, index_col=0)
    
    exact_matrix_features = np.array([features_to_values[feature] for feature in sliced_matrix_index.values.flatten()])\
                        .reshape(sliced_matrix_index.shape).astype(np.float32)
        
    return exact_matrix_features

def get_matrix_features_by_str_with_w2v(
                string, 
                word_to_ind_mapping, 
                matrix_index_path, 
                mapping_input_str_path,
                lemma_file, 
                pos_file,
                lemma_ind,
                pos_ind):
    
    # Word to ind mapping for lemma_pos representation
    word_to_ind_mapping = pickle.load(open(word_to_ind_mapping, 'rb'))
    # All features in the input string
    mapping_input_str = open(mapping_input_str_path).readlines()
    mapping_input_str = [x[:-2] for x in mapping_input_str] # delete \r \n symbols
    
    # Map features to their exact values in the input string
    features_to_values = {k: v for k, v in zip(mapping_input_str, string.split('\t'))}
    
    # This is previously f1_score matrix. Here i use it to infer of the order of features in the matrix
    sliced_matrix_index = pd.read_csv(matrix_index_path, index_col=0)
    
    # Parse input string to matrix format (without word2vec)
    exact_matrix_features = np.array([features_to_values[feature] for feature in sliced_matrix_index.values.flatten()])\
                        .reshape(sliced_matrix_index.shape).astype(np.float32)
    
    # Get lemma_pos for created matrix
    lemma_pos, _, _ = get_lemma_pos_representation_for_single_sample(matrix_features, \
                                        lemma_file, pos_file, lemma_ind, pos_ind)
    
    # Get w2v features for the matrix using word to ind mapping
    w2v_features = np.array([word_to_ind_mapping[word] for word in lemma_pos]).reshape((len(lemma_pos),1)) 
    
    # return a matrix with w2v features
    return np.concatenate((exact_matrix_features, w2v_features), axis=1)

In [8]:
matrix_features = get_matrix_features_by_str_without_w2v(string,
                          '/home/sag/syntactic/src_putin/grant/data/raw/sliced_matrix_index.csv',
                          '/home/sag/syntactic/src_putin/grant/data/raw/mapping_input_str.txt')

matrix_features_w2v = get_matrix_features_by_str_with_w2v(string,
                          SELECTED_FEATURES+"word_to_ind_mapping.pkl",
                          '/home/sag/syntactic/src_putin/grant/data/raw/sliced_matrix_index.csv',
                          '/home/sag/syntactic/src_putin/grant/data/raw/mapping_input_str.txt',
                          SELECTED_FEATURES+"Lemma.txt", SELECTED_FEATURES+"pos.txt", LEMMA_IND, POS_IND)

print(matrix_features.shape, matrix_features_w2v.shape)

((13, 12), (13, 13))


In [9]:
model_without_w2v_features = load_keras_model(SELECTED_FEATURES+"model_without_w2v_init.pkl")
model_with_w2v_features = load_keras_model(SELECTED_FEATURES+"model_with_w2v_init.pkl")

In [10]:
first = [elem[0].reshape((1,X_val[0].shape[1])) for elem in X_val]
first_w2v = [elem[0].reshape((1,X_val_w2v[0].shape[1])) for elem in X_val_w2v]

pred = model_without_w2v_features.predict(first)
pred_w2v = model_with_w2v_features.predict(first_w2v)
#pred = model_with_w2v_features.predict(first)
print(pred.shape, pred_w2v.shape)

for dim in first:
    print(dim.shape)
print(len(first))

for dim in first_w2v:
    print(dim.shape)
print(len(first_w2v))

((1, 125), (1, 125))


In [14]:
to_model = [matrix_features[i,:].reshape((1,matrix_features.shape[1])) for i in range(matrix_features.shape[1])]
to_model_w2v = [matrix_features_w2v[i,:].reshape((1,matrix_features_w2v.shape[1])) for i in range(matrix_features_w2v.shape[1])]

for dim in to_model:
    print(dim.shape)
print(len(to_model))

for dim in to_model_w2v:
    print(dim.shape)
print(len(to_model_w2v))

In [20]:
pred = model_with_w2v_features.predict(to_model_w2v)

IndexError: One of the index value is out of bound. Error code: 65535.\n
Apply node that caused the error: GpuAdvancedSubtensor1(embedding_16_W, Elemwise{Cast{int64}}.0)
Toposort index: 121
Inputs types: [CudaNdarrayType(float32, matrix), TensorType(int64, vector)]
Inputs shapes: [(6, 3), (13,)]
Inputs strides: [(3, 1), (8,)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuReshape{3}(GpuAdvancedSubtensor1.0, MakeVector{dtype='int64'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [19]:
pred = model_without_w2v_features.predict(to_model)

Exception: Error when checking : expected embedding_input_14 to have shape (None, 13) but got array with shape (1, 12)